**Задание Pro**

Добейтесь точности распознавания 97% и верно распознанных всех писателей с помощью любой нейронной сети без фильтрации данных. Попробуйте менять размер окна и шаг

# Подключение библиотек 

In [1]:
# Для начала работы подключим все необходимые библиотеки
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense,  BatchNormalization, Activation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import utils
from google.colab import drive
import numpy as np
from google.colab import files
import gc 


# Объявим все необходимые функции

In [2]:
# Создадим функцию для первоначальной обработки файла
def readText(fileName):
  op = open(fileName, 'r')        # Указываем что открытие будет в режиме чтения
  text = op.read()                # Читаем файл с текстом
  text = text.replace("\n", " ")  # Заменям переносы строк на пробелы
  return text                     # Возвращаем измененный файл

In [3]:
# Для начала создадим функции. 
# Первая для разделения на короткие векторы с заданным шагом и размером

def getSetFromIndexes (wordIndexes, xLen, step):     # В качестве аргументов принимает (trainWordIndexes или testWordIndexes - список, размер и шаг) 
  xSample = []                                        # Созадим пустой спискок для хранения векторов
  wordsLen = len(wordIndexes)                         # Кол-во слов в поданных на вход trainWordIndexes или testWordIndexes
  index = 0                                           # Начальный индекс (с помощью этой переменной будем сдвигать шаг)


  while (index + xLen <= wordsLen):                   # Задаем цикл пока не дойдем до конца    
    xSample.append(wordIndexes[index:index + xLen])   # На каждой итерации добавляем в список кусок вектора заданной длины
    index += step                                     # Смещаемся на шаг

  return xSample

In [4]:
# Вторая функция для формирования обучающей и проверочной выборок

def createSetsMultiClasses(wordIndexes, xLen, step):
  nClasses = len(wordIndexes)                            # Зададим кол-во классов выборки
  classesXSample = []                                    # Создадим пустой список в котором будут храниться список нарезанных кусочков предыдущей функцией getSetFromIndexes

  # Пройдемся циклом по каждому тексту выборки из последовательнстей индексов
  for i in wordIndexes:
    classesXSample.append(getSetFromIndexes(i, xLen, step))
  
  # Сформируем один общий список
  xSamples = []                                           # Создаем пустые списки
  ySamples = []

  for t in range(nClasses):                               # В нашем случаи цикл от 0 до 6 (кол-во классов)
    xT = classesXSample[t]                                # На каждой итерации цикла берем очередной нарезанный текст одного из класса

    for i in range (len(xT)):                             # Циклом проходимся по каждому его окну
      xSamples.append(xT[i])                              # Добавляем в общий список обучающей выборке
      ySamples.append(utils.to_categorical(t, nClasses))  # И соответствующий вектор класса
  
  # Переводим в numpy массив
  xSamples = np.array(xSamples)                           
  ySamples = np.array(ySamples)    

  # B возвращаем выборки
  return (xSamples, ySamples)

In [5]:
# Создадим функцию которая будет представлять выборку в удобном для распознавания размерах

def createTestMultiClasses(wordIndexes, xLen, step):

  # Создаем тестовую выборку из индексов для каждого из 6 классов
  nClasses = len(wordIndexes)                                             # Переменной nClasses присвоим кол-во классов
  xTest6ClassesBow = []                                                   # Создадим пустой список, в котором будут список из всех классов
  xTest6Classec = []                                                      # Создадим список массивов

  # Пройдемся циклом по каждому тестовому тексту из последовательности индексов
  for i in wordIndexes:
    sample = (getSetFromIndexes(i,xLen, step))                            # Воспользуемся ранее написанной функцией для нарезки на векторы
    xTest6Classec.append(sample)                                          # Добавляем в список на каждой итерации 
    xTest6ClassesBow.append(tokenizer.sequences_to_matrix(sample))        # Трансформируем в BOW
  
  xTest6ClassesBow = np.array(xTest6ClassesBow)                           # Переводим в numpy массив
  xTest6Classec = np.array(xTest6Classec)                                 # Переводим в numpy массив

  return xTest6ClassesBow, xTest6Classec

In [6]:
# Создадим еще одну функцию которая будет распознавать тестовую выборку и выводить результаты
def recognizerMultiClass (model, xTest, modelName):
  #print ("Название нейронной сети: ", modelName)        # Для наглядности выведем название сети
  #print ()                                              # Пропустим стороку
  totalSumRec = 0                                       # Создадим счетчик суммы правильных ответов

  # Пройдемся циклом по всем классам
  for i in range(nClasses):                             # От 0 до 6 
    currPred = model.predict(xTest[i])                  # Получим результаты распознования класса по блокам слов длинны xLen.
    currOut = np.argmax(currPred, axis=1)               # Определим номер распознанного класса с помощью функции argmax
    evVal = []
    
    for j in range(nClasses):
      evVal.append(len(currOut[currOut==j])/len(xTest[i]))

    totalSumRec +=len(currOut[currOut==i])
    recognizedClass = np.argmax(evVal)                  # Определяем какой класс за какой был распознан в итоге

    # И выводим результаты

    isRecognized = "Ответ НЕВЕРНЫЙ, УВЫ"
    if (recognizedClass == i):
      isRecognized = "УРА, ответ ВЕРНЫЙ!"
    
    str1 = 'Класс: ' + className[i] + " " * (11 - len(className[i])) + str(int(100*evVal[i])) + "% сеть предсказала: " + className[recognizedClass]
    print(str1, " " * (55-len(str1)), isRecognized, sep='')

  # Выведим средний процент распознавания 
  print ()                                                                          # Пустая строка чтобы отделить
  sumCount = 0
  # Пройдемся циклом по всем циклам
  for i in range(nClasses):
    sumCount += len(xTest[i])
  print ("Средний процент распознавания ", int(100*totalSumRec/sumCount), "%")
  print ()                                                                          # Пустая строка чтобы отделить

  return totalSumRec/sumCount

# Подготовка данных

In [7]:
# Объявим классы. Запишем в список
className = ["О. Генри", "Стругацкие", "Булгаков", "Саймак", "Фрай", "Брэдберри"]

# Подсчитаем кол-во нужных нам классов
nClasses = len(className)

# Создадим 2 пустых списка (обучающий и проверочный), для последующего заполнения
trainText = []
testText = []

In [8]:
# Пройдемся двумя циклами для заполнения списков. Так чтобы каждый i элемент в обучающей выборке соответствовал j элементу в тестовой.

for i in className:
  for j in os.listdir('/content/'):                     
    if i in j:                                       
      if 'Обучающая' in j:                           
        trainText.append(readText('/content/' +j))      
        print (j, 'добавлен в обучающую выборку')    
      if 'Тестовая' in j:                            
        testText.append(readText('/content/' +j))       
        print (j, 'добавлен в тестовую выборку')     

(О. Генри) Тестовая_20 вместе.txt добавлен в тестовую выборку
(О. Генри) Обучающая_50 вместе.txt добавлен в обучающую выборку
(Стругацкие) Тестовая_2 вместе.txt добавлен в тестовую выборку
(Стругацкие) Обучающая_5 вместе.txt добавлен в обучающую выборку
(Булгаков) Обучающая_5 вместе.txt добавлен в обучающую выборку
(Булгаков) Тестовая_2 вместе.txt добавлен в тестовую выборку
(Клиффорд_Саймак) Тестовая_2 вместе.txt добавлен в тестовую выборку
(Клиффорд_Саймак) Обучающая_5 вместе.txt добавлен в обучающую выборку
(Макс Фрай) Тестовая_2 вместе.txt добавлен в тестовую выборку
(Макс Фрай) Обучающая_5 вместе.txt добавлен в обучающую выборку
(Рэй Брэдберри) Обучающая_22 вместе.txt добавлен в обучающую выборку
(Рэй Брэдберри) Тестовая_8 вместе.txt добавлен в тестовую выборку


In [9]:
# Зададим значения которые будут неизменными во всем ноутбуке.

maxWordsCount = 15000
xLen = 5000                              # Длина отрезка текста на которые будет разбивать функция getSetFromIndexes
step = 100                               # Шаг разбивания для той же функции

**Так как в заднии сказано без фильтрации данных, токенизировать будем без filters**.

In [10]:
tokenizer = Tokenizer(num_words = maxWordsCount, lower=True, split=' ', oov_token ='unknow', char_level = False)
tokenizer.fit_on_texts(trainText)   
items = list(tokenizer.word_index.items())

# Согласно частотному словарю можно преобразовать текст в последовательность индексов
trainWordIndexes = tokenizer.texts_to_sequences(trainText)
testWordIndexes = tokenizer.texts_to_sequences(testText)   

# Установим базовые параметры

# Сформируем выборки (обучающую и тестовую)
xTrain, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
xTest, yTest = createSetsMultiClasses(testWordIndexes, xLen, step)

# Преобразовываем в матрицы нулей и единиц по принципу BOW. xTrain и yTrain подаем в виду списка. 
xTrainBOW = tokenizer.sequences_to_matrix(xTrain.tolist())      
xTestBOW = tokenizer.sequences_to_matrix(xTest.tolist()) 

По сравнению с прошлым заданием ноутбук не на столько большой. Но подготовительной работы было много. Так как найти нужную архитектуру нейронной сети и подобрать парвильные гиперпараметры, чтобы добиться определенного результата было не просто. Изменялись такие параметры:
  - Значение **maxWordsCount** в пределах от 5000 до 25000
  - Длина отрезка текста на которые будет разбивать функция от 500 до 5000
  - Шаг от 50 до 500
  - Количество скрытых слоев:
    - Добавление еще полносвязных **Dense** слоев
    - Добавление слоев нормализации **BatchNormalization**
    - Добавление слоя **Dropout**
  - Изменение количетсва нейронов в **Dense** слоях в пределах от 10 до 1000
  - Изменение параметра в слоях **Dropout** 
  - Изменения активационных функций
  - Изменения шага обучения **lr**


Оставил только нейронные сети, которые удовлетворяют показанию на тестовой выборке больше 97%, остальных вариантов было много и чтобы не засорять ноутбук в работу не включил. 

In [11]:
# чистим оперативную память   
gc.collect()

15

# Вариант 1

Первый раз добился результата с данной архитектурой. Но пришлось обучать данную сеть 2 раза. Первый раз с шагом обучения по умолчания lr=0.001,
а второй раз уже с шагом lr=0.0001. 

Но зато результат больше 97%.

In [12]:
# Создадим полносвязную сеть
modelBow = Sequential()
modelBow.add(Dense(300, input_dim=maxWordsCount, activation = 'sigmoid'))   
modelBow.add(Dropout(0.5))
modelBow.add(BatchNormalization())

modelBow.add(Dense(600, input_dim=maxWordsCount, activation = 'relu'))   
modelBow.add(Dropout(0.5))
modelBow.add(BatchNormalization())

modelBow.add(Dense(6, activation ='softmax'))

# Скомпилируем ее
modelBow.compile(optimizer = Adam, loss='categorical_crossentropy', metrics=['accuracy'])

# Обучаем сеть на выборке, xTrainBOW
history = modelBow.fit(xTrainBOW, yTrain, epochs=15, batch_size=150, validation_data=(xTestBOW, yTest), verbose=1)

Epoch 1/15
122/122 [==============================] - 4s 15ms/step - loss: 0.3143 - accuracy: 0.8957 - val_loss: 0.4156 - val_accuracy: 0.9515
Epoch 2/15
122/122 [==============================] - 1s 10ms/step - loss: 8.4462e-04 - accuracy: 1.0000 - val_loss: 0.2494 - val_accuracy: 0.9477
Epoch 3/15
122/122 [==============================] - 1s 10ms/step - loss: 4.6049e-04 - accuracy: 1.0000 - val_loss: 0.1862 - val_accuracy: 0.9536
Epoch 4/15
122/122 [==============================] - 1s 10ms/step - loss: 3.8021e-04 - accuracy: 1.0000 - val_loss: 0.1705 - val_accuracy: 0.9456
Epoch 5/15
122/122 [==============================] - 1s 10ms/step - loss: 2.0022e-04 - accuracy: 1.0000 - val_loss: 0.1504 - val_accuracy: 0.9570
Epoch 6/15
122/122 [==============================] - 1s 10ms/step - loss: 1.6134e-04 - accuracy: 1.0000 - val_loss: 0.1405 - val_accuracy: 0.9584
Epoch 7/15
122/122 [==============================] - 1s 10ms/step - loss: 1.2379e-04 - accuracy: 1.0000 - val_loss: 0.138

**Снова обучим но с шагом lr = 0.0001**

In [13]:
# Скомпилируем ее
modelBow.compile(optimizer=Adam(lr = 0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Обучаем сеть на выборке, xTrainBOW
history = modelBow.fit(xTrainBOW, yTrain, epochs=15, batch_size=150, validation_data=(xTestBOW, yTest), verbose=1)

Epoch 1/15
122/122 [==============================] - 2s 13ms/step - loss: 2.1601e-05 - accuracy: 1.0000 - val_loss: 0.1014 - val_accuracy: 0.9596
Epoch 2/15
122/122 [==============================] - 1s 10ms/step - loss: 9.0258e-06 - accuracy: 1.0000 - val_loss: 0.0901 - val_accuracy: 0.9636
Epoch 3/15
122/122 [==============================] - 1s 11ms/step - loss: 8.8391e-06 - accuracy: 1.0000 - val_loss: 0.0917 - val_accuracy: 0.9611
Epoch 4/15
122/122 [==============================] - 1s 10ms/step - loss: 8.6125e-06 - accuracy: 1.0000 - val_loss: 0.0902 - val_accuracy: 0.9581
Epoch 5/15
122/122 [==============================] - 1s 10ms/step - loss: 5.2403e-06 - accuracy: 1.0000 - val_loss: 0.0747 - val_accuracy: 0.9716
Epoch 6/15
122/122 [==============================] - 1s 10ms/step - loss: 3.5276e-06 - accuracy: 1.0000 - val_loss: 0.0767 - val_accuracy: 0.9694
Epoch 7/15
122/122 [==============================] - 1s 10ms/step - loss: 4.6802e-06 - accuracy: 1.0000 - val_loss: 0

In [14]:
# Распознавание проверочной выборки

# Преобразуем тестовую выборку
xTest6ClassesBow, x2 = createTestMultiClasses(testWordIndexes, xLen, step)

# Проверим точность сети на BOW
pred = recognizerMultiClass (modelBow, xTest6ClassesBow, "BOW: Dense-DropOut-BatchNormalization-Dense")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Класс: О. Генри   100% сеть предсказала: О. Генри      УРА, ответ ВЕРНЫЙ!
Класс: Стругацкие 94% сеть предсказала: Стругацкие     УРА, ответ ВЕРНЫЙ!
Класс: Булгаков   93% сеть предсказала: Булгаков       УРА, ответ ВЕРНЫЙ!
Класс: Саймак     100% сеть предсказала: Саймак        УРА, ответ ВЕРНЫЙ!
Класс: Фрай       97% сеть предсказала: Фрай           УРА, ответ ВЕРНЫЙ!
Класс: Брэдберри  100% сеть предсказала: Брэдберри     УРА, ответ ВЕРНЫЙ!

Средний процент распознавания  97 %



In [17]:
# Сохраним полученные веса
modelBow.save_weights('loss: 7.1519e-07 - accuracy: 1.0000 - val_loss: 0.0685 - val_accuracy: 0.9712.h5')

Ура ! Получилось. Такого результата удалось добиться изменив значения: 

maxWordsCount = **15000**

xLen = **5000**

step = **100 **          

Теперь можно попробовать упростить архитектуру нейронной сети. Посмотрим что получится. 
   Уберем часть скрытых слоев и оставим всего 2.

In [18]:
# Создадим полносвязную сеть
modelBow = Sequential()
modelBow.add(Dense(300, input_dim=maxWordsCount, activation = 'sigmoid'))   
modelBow.add(Dropout(0.5))
modelBow.add(BatchNormalization())

#modelBow.add(Dense(600, input_dim=maxWordsCount, activation = 'relu'))   
#modelBow.add(Dropout(0.5))
#modelBow.add(BatchNormalization())

modelBow.add(Dense(6, activation ='softmax'))

# Скомпилируем ее
modelBow.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучаем сеть на выборке, xTrainBOW
history = modelBow.fit(xTrainBOW, yTrain, epochs=15, batch_size=150, validation_data=(xTestBOW, yTest), verbose=1)

Epoch 1/15
122/122 [==============================] - 2s 13ms/step - loss: 0.1799 - accuracy: 0.9423 - val_loss: 0.3394 - val_accuracy: 0.9639
Epoch 2/15
122/122 [==============================] - 1s 10ms/step - loss: 3.1406e-04 - accuracy: 1.0000 - val_loss: 0.2111 - val_accuracy: 0.9639
Epoch 3/15
122/122 [==============================] - 1s 9ms/step - loss: 1.7381e-04 - accuracy: 1.0000 - val_loss: 0.1724 - val_accuracy: 0.9633
Epoch 4/15
122/122 [==============================] - 1s 9ms/step - loss: 1.3758e-04 - accuracy: 1.0000 - val_loss: 0.1572 - val_accuracy: 0.9642
Epoch 5/15
122/122 [==============================] - 1s 9ms/step - loss: 1.0093e-04 - accuracy: 1.0000 - val_loss: 0.1496 - val_accuracy: 0.9663
Epoch 6/15
122/122 [==============================] - 1s 10ms/step - loss: 7.5727e-05 - accuracy: 1.0000 - val_loss: 0.1436 - val_accuracy: 0.9674
Epoch 7/15
122/122 [==============================] - 1s 9ms/step - loss: 6.5211e-05 - accuracy: 1.0000 - val_loss: 0.1407 - 

In [19]:
# Распознавание проверочной выборки

# Преобразуем тестовую выборку
xTest6ClassesBow, x2 = createTestMultiClasses(testWordIndexes, xLen, step)

# Проверим точность сети на BOW
pred = recognizerMultiClass (modelBow, xTest6ClassesBow, "BOW: Dense-DropOut-BatchNormalization-Dense")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Класс: О. Генри   100% сеть предсказала: О. Генри      УРА, ответ ВЕРНЫЙ!
Класс: Стругацкие 91% сеть предсказала: Стругацкие     УРА, ответ ВЕРНЫЙ!
Класс: Булгаков   100% сеть предсказала: Булгаков      УРА, ответ ВЕРНЫЙ!
Класс: Саймак     100% сеть предсказала: Саймак        УРА, ответ ВЕРНЫЙ!
Класс: Фрай       95% сеть предсказала: Фрай           УРА, ответ ВЕРНЫЙ!
Класс: Брэдберри  100% сеть предсказала: Брэдберри     УРА, ответ ВЕРНЫЙ!

Средний процент распознавания  97 %



Получилось добиться такого же результа с меньшим количество скрытых слоев и обучая нейронную сеть один раз. С шагом обучения lr=0.001 

In [20]:
# Сохраним полученные веса
modelBow.save_weights('loss: 2.1834e-05 - accuracy: 1.0000 - val_loss: 0.1186 - val_accuracy: 0.9731.h5')

# Выводы:

1. Данного результат удалось добиться путем многих десятков экспериментов.
2. Очень помогло увелить **xLen** -  Длина отрезка текста на которые будет разбивать функция. При значениях до **1000**(100,200,500) идет переполнение памяти. От** 1000** до **3000** результат в районе **90%**. Свыше **4000** результат намного лучше.
3. Так же зависит и шаг разбивания. Если делать маленьким, то так же будет переполнение памяти. Слишком большой тоже влияет на результат в худшую сторону. Оптимально оказалось значение **100**.
4. Нельзя не сказать о maxWordsCount. Слишком маленький словарь, меньше **1000** значительно ухудшает предстказание нейронной сети. В тоже время значение выше **25000** тоже не приводят к хорошему результату, даже наоборот. Оптимальное значенеи в пределах от **10000** до **20000**. 
5. Количество нейронов с полносвязном слое **Dense** тоже вносят свои коррективы. Слишком большое кол-во нейронов не около **300**.
6. Так же как и кол-во скрытых слоев. Как видно с меньшим количеством нейронная сеть показала результат ничуть не хуже, чем с большим им числом.
7. Значение слоя **Dropout** тоже подбиралось в хоже экспериментов. Слишком малое значение не приводило к удучшению результата. Оптимально в данном случаи оказалось значение **0.5**. 
8. Выбор активациооной функции также повлияло на результат. Функция **sigmoid** в данной сети зарекомендовала себя лучше чем **relu** и другие. 
9. В купе выбор оптимальной архитектуры и подбор всех сопутствующих гиперпараметров дали хороший результат, который удовлетворяет условию задания. 
10. Выбор архитектуры и подбор гиперпаметров требуют опыта, который достигается благодаря исполнению подобных заданий. 